# BentoML을 이용해서 서빙 & 배포하기

Service, Docker, Deployment

### Reference

[Kaggle Notebook: Natural Language Processing (NLP) for Beginners](https://www.kaggle.com/code/faressayah/natural-language-processing-nlp-for-beginners#%F0%9F%94%8D-Exploratory-Data-Analysis-(EDA)) <br>
[Spam Filter in Python: Naive Bayes from Scratch](https://www.kdnuggets.com/2020/07/spam-filter-python-naive-bayes-scratch.html) <br>
[Naive Bayes, Clearly Explained!!!](https://www.youtube.com/watch?v=O2L2Uv9pdDA&t=269s) <br>
[Picklable Model in BentoML](https://docs.bentoml.org/en/latest/frameworks/picklable.html) <br>



In [3]:
!pip install pandas
!pip install -U bentoml

In [4]:
import bentoml
import pickle
f = open('../models/nb_model.pickle', 'rb')
classifier = pickle.load(f)
# Save a model in Bentoml Store
saved_model = bentoml.picklable_model.save_model("spam_detector", classifier)
print(f"Model saved: {saved_model}")

Model saved: Model(tag="spam_detector:pnluevye32tgx3c4")


In [5]:
!bentoml models list
!bentoml models get spam_detector:latest

 Tag                    Module                 Size        Creation Time       
 spam_detector:pnluev���  bentoml.picklable_mo���  426.37 KiB  2022-07-16 04:08:31 
 spam_detector:uub75a���  bentoml.picklable_mo���  426.38 KiB  2022-07-12 20:35:07 
 spam_detector:2x57ov���  bentoml.picklable_mo���  451.00 B    2022-07-12 20:22:10 
 spam_detector:roz7cm���  bentoml.picklable_mo���  451.00 B    2022-07-12 02:26:21 
name: spam_detector                                                            
version: pnluevye32tgx3c4                                                      
module: bentoml.picklable_model                                                
labels: {}                                                                     
options: {}                                                                    
metadata: {}                                                                   
context:                                                                       
  framework_name: cloudp

## Serving the model

A simple BentoML Service that serves the model saved above look like this:

In [6]:
%%writefile nb_service.py
import bentoml
import logging
import re
from bentoml.io import Text, JSON

bentoml_logger = logging.getLogger("bentoml")

class spamdetectionrunnable(bentoml.Runnable):
    SUPPORTED_RESOURCES = ()
    SUPPORTS_CPU_MULTI_THREADING = False

    def __init__(self):
        # load the model back:
        classifier = bentoml.picklable_model.load_model("spam_detector:latest")
        self.p_spam, self.p_ham, self.parameters_spam, self.parameters_ham = classifier
        
    @bentoml.Runnable.method(batchable=False)
    def is_spam (self, input_text):

        message = re.sub('\W', ' ', input_text)
        message = message.lower().split()

        p_spam_given_message = self.p_spam
        p_ham_given_message = self.p_ham
       
        for word in message:
            if word in self.parameters_spam:
                p_spam_given_message *= self.parameters_spam[word]

            if word in self.parameters_ham:
                p_ham_given_message *= self.parameters_ham[word]

        if p_ham_given_message < p_spam_given_message:
            return [True, p_spam_given_message, p_ham_given_message]
        else:
            return [False, p_spam_given_message, p_ham_given_message]
        

spam_detection_runner = bentoml.Runner(spamdetectionrunnable, models=[bentoml.models.get("spam_detector:latest")])
svc = bentoml.Service('spam_classify', runners=[spam_detection_runner])

@svc.api(input=Text(), output=JSON())
def analysis(input_text):
    is_spam, p_spam_given_message,  p_ham_given_message= spam_detection_runner.is_spam.run(input_text)
    return { "is_spam": is_spam, "P(Spam|message)": p_spam_given_message, "P(Ham|message)": p_ham_given_message}

Writing nb_service.py


# service.py 설명
Note: using `%%writefile` here because `bentoml.Service` definition must be created in its own `.py` file

Start a dev model server to test out the service defined above:

In [7]:
!bentoml serve nb_service.py:svc

# 참고로, AWS에서는 host 를 써준다. 여기서 Host는 AWS의 EC2의 프라이빗 IPv4 주소
#!bentoml serve nb_service.py:svc --host 172.31.90.78

^C


1. 웹 브라우저 사용해서 확인할 수 있다. http://127.0.0.1:3000  (AWS EC2를 외부 브라우저로 접속할 경우, 퍼블릭 IPv4 주소를 127.0.0.1 대신 사용한다) 로 연결해서 녹색으로 된 app POST 섹션을 클릭한다. "Try it out" 클릭한 후에 string 부분을 원하는 문장으로 넣고, execute 를 하면 된다.

AWS EC2 에서 serving 할 때, 이를 외부에서 확인하기 위해서는, 3000 번 port 를 보안탭에서 inbound 규칙에 추가한다.
포트 범위: 3000
프로토콜: TCP
원본: 0.0.0.0/0


2. Curl 사용해서 할 수 있다. e.g.:

curl -X POST -H "content-type: application/json" --data "WINNER!! This is the secret code to unlock the money: C3421." http://127.0.0.1:3000/analysis

참고. content-type 에 대해서는 웹 브라우저 있는데로 text/plain 으로 바꿔도 잘 됨. 어떤 것이 많는지 확인할 필요 있을 수도...

curl -X POST -H "content-type: text/plain" --data "WINNER!! This is the secret code to unlock the money: C3421." http://127.0.0.1:3000/analysis

## 도커로 배포하기

참고적으로, 다음과 같은 코드는 BentoML에서 path 설정시 오류가 있었던 버전이 있어서 설정하였다.
docker:
    system_packages:
        - git
...        
    - git+https://github.com/bentoml/BentoML.git@main
    
    
Bento is the distribution format in BentoML which captures all the source code, model files, config
files and dependency specifications required for running the service for production deployment. Think 
of it as Docker/Container designed for machine learning models.

To begin with building Bento, create a `bentofile.yaml` under your project directory:

In [8]:
%%writefile bentofile.yaml
service: "nb_service.py:svc"
labels:
  owner: bentoml-team
  project: gallery
include:
- "*.py"
python:
  packages:
    - pandas

Overwriting bentofile.yaml


In [4]:
%%writefile bentofile.yaml
docker:
    system_packages:
        - git
service: "nb_service.py:svc"
labels:
  owner: bentoml-team
  project: gallery
include:
- "*.py"
python:
  packages:
    - pandas
    - git+https://github.com/bentoml/BentoML.git@main

Overwriting bentofile.yaml


Next, run bentoml build from current directory to start the Bento build:

In [9]:
!bentoml build

Building BentoML service "spam_classify:zqk4z3ye32lnz3c4" from build context "C:\Users\hwang\OneDrive\Documents\GitHub\MLOpsToyProject\notebooks"
Packing model "spam_detector:pnluevye32tgx3c4"
Locking PyPI package versions..

�뻽�뻽�뻽�뻽�뻽�뻽�븮�뼇�뻽�뻽�뻽�뻽�뻽�뻽�뻽�븮�뻽�뻽�뻽�븮�뼇�뼇�뻽�뻽�븮�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�븮�뼇�뻽�뻽�뻽�뻽�뻽�븮�뼇�뻽�뻽�뻽�븮�뼇�뼇�뼇�뻽�뻽�뻽�븮�뻽�뻽�븮�뼇�뼇�뼇�뼇�뼇
�뻽�뻽�븫�븧�븧�뻽�뻽�븮�뻽�뻽�븫�븧�븧�븧�븧�븴�뻽�뻽�뻽�뻽�븮�뼇�뻽�뻽�븨�븱�븧�븧�뻽�뻽�븫�븧�븧�븴�뻽�뻽�븫�븧�븧�뻽�뻽�븮�뻽�뻽�뻽�뻽�븮�뼇�뻽�뻽�뻽�뻽�븨�뻽�뻽�븨�뼇�뼇�뼇�뼇�뼇
�뻽�뻽�뻽�뻽�뻽�뻽�븽�븴�뻽�뻽�뻽�뻽�뻽�븮�뼇�뼇�뻽�뻽�븫�뻽�뻽�븮�뻽�뻽�븨�뼇�뼇�뼇�뻽�뻽�븨�뼇�뼇�뼇�뻽�뻽�븨�뼇�뼇�뻽�뻽�븨�뻽�뻽�븫�뻽�뻽�뻽�뻽�븫�뻽�뻽�븨�뻽�뻽�븨�뼇�뼇�뼇�뼇�뼇
�뻽�뻽�븫�븧�븧�뻽�뻽�븮�뻽�뻽�븫�븧�븧�븴�뼇�뼇�뻽�뻽�븨�븱�뻽�뻽�뻽�뻽�븨�뼇�뼇�뼇�뻽�뻽�븨�뼇�뼇�뼇�뻽�뻽�븨�뼇�뼇�뻽�뻽�븨�뻽�뻽�븨�븱�뻽�뻽�븫�븴�뻽�뻽�븨�뻽�뻽�븨�뼇�뼇�뼇�뼇�뼇
�뻽�뻽�뻽�뻽�뻽�뻽�븽�븴�뻽�뻽�뻽�뻽�뻽�뻽�뻽�븮�뻽�뻽�븨�뼇�븱�뻽�뻽�뻽�븨�뼇�뼇�뼇�뻽�뻽�븨�뼇�뼇�뼇�븱�뻽�뻽�뻽�뻽�뻽�븫�븴�뻽�뻽�븨�뼇�븱�븧�븴�뼇�뻽�뻽�븨�뻽�뻽�뻽�뻽�뻽�뻽�뻽�븮
�븱�븧�븧�븧�븧�븧�븴�뼇�븱�븧�븧�븧�븧�븧�븧�븴�븱�븧�븴�뼇�뼇�븱�븧�븧�븴�뼇�뼇�뼇�븱�븧�븴�뼇�뼇�뼇�뼇�븱�븧�븧�븧�븧�븴�뼇�븱�븧�븴�뼇�뼇�뼇�뼇�뼇�븱�븧�븴�븱�븧�븧�븧�븧�븧�븧�븴

Successfully built Bento(tag="spam_

Using lowercased runnable class name 'spamdetectionrunnable' for runner.


A new Bento is now built and saved to local Bento store. You can view and manage it via 
`bentoml list`,`bentoml get` and `bentoml delete` CLI command.

## 도커 컨테이너 만들기

1. Docker Desktop 다운로드 받아서 설치한 후 Docker daemon 실행. Docker Desktop 홈페이지 https://www.docker.com/products/docker-desktop/
2. Bentoml 명령어로 도커 컨테이너 만든다
bentoml containerize spam_detector:latest
    
3. Command line에서 Docker 이미지가 만들어졌는지 확인한다.
docker images

4. Command line에서 Docker 를 실행한다. 참고적으로 두의 tag는 위에 build 할 때 버전을 참고한다.
docker run -p 3000:3000 spam_detector:eesmbbyar6db73c4

In [12]:
!bentoml containerize spam_classify:latest

Building docker image for Bento(tag="spam_classify:zqk4z3ye32lnz3c4")...
Successfully built docker image "spam_classify:zqk4z3ye32lnz3c4"


#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2.24kB done
#1 DONE 0.1s

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.1s

#3 resolve image config for docker.io/docker/dockerfile:1.4-labs
#3 DONE 1.2s

#4 docker-image://docker.io/docker/dockerfile:1.4-labs@sha256:b50ad4af81d1c76ab7c0e1ffc216909e7adc23e99910243e1c88331c2a8ef52d
#4 CACHED

#5 [internal] load .dockerignore
#5 DONE 0.0s

#6 [internal] load build definition from Dockerfile
#6 DONE 0.0s

#7 [internal] load metadata for docker.io/library/python:3.9-slim
#7 DONE 0.6s

#8 [stage-0  1/11] FROM docker.io/library/python:3.9-slim@sha256:ea93ec4fbe8ee1c62397410c0d1f342a33199e98cd59adac6964b38e410e8246
#8 resolve docker.io/library/python:3.9-slim@sha256:ea93ec4fbe8ee1c62397410c0d1f342a33199e98cd59adac6964b38e410e8246 0.0s done
#8 DONE 0.0s

#9 [internal] load build context
#9 transferring context: 449.38kB 0.1s done
#9 DONE 0.1s

#10 [stage-0  2/11] RUN rm -f /etc/apt/apt

In [13]:
# Docker Run을 실행한다. "ip6hyaybqktr33c4" 는 위에서 만들어진 도커 컨테이너 태그로 수정 요망
!docker run -p 3000:3000 spam_classify:zqk4z3ye32lnz3c4

2022-07-16T08:12:33+0000 [WARNING] [cli] Using lowercased runnable class name 'spamdetectionrunnable' for runner.
2022-07-16T08:12:33+0000 [INFO] [cli] Service loaded from Bento directory: bentoml.Service(tag="spam_classify:zqk4z3ye32lnz3c4", path="/home/bentoml/bento/")
2022-07-16T08:12:33+0000 [WARNING] [cli] No known supported resource available for <class 'nb_service.spamdetectionrunnable'>, falling back to using CPU.
2022-07-16T08:12:34+0000 [INFO] [cli] Starting production BentoServer from "/home/bentoml/bento" running on http://0.0.0.0:3000 (Press CTRL+C to quit)
2022-07-16T08:12:35+0000 [WARNING] [api_server:3] Using lowercased runnable class name 'spamdetectionrunnable' for runner.
2022-07-16T08:12:35+0000 [WARNING] [api_server:2] Using lowercased runnable class name 'spamdetectionrunnable' for runner.
2022-07-16T08:12:35+0000 [INFO] [api_server:3] Service loaded from Bento directory: bentoml.Service(tag="spam_classify:zqk4z3ye32lnz3c4", path="/home/bentoml/bento/")
2022-07-16

!Bento is designed to be deployed to run efficiently in a variety of different environments.
And there are lots of deployment options and tools as part of the BentoML eco-system, such as 
[Yatai](https://github.com/bentoml/Yatai) and [bentoctl](https://github.com/bentoml/bentoctl) for
direct deployment to cloud platforms.

In this guide, we will show you the most basic way of deploying a Bento, which is converting a Bento
into a Docker image containing the HTTP model server.

Make sure you have docker installed and docker deamon running, and run the following commnand:

```bash
bentoml containerize spam_classify:latest
```

This will build a new docker image with all source code, model files and dependencies in place,
and ready for production deployment. To start a container with this docker image locally, run:

```bash
docker run -p 3000:3000 spam_classify:ip6hyaybqktr33c4
```

## What's Next?

- 👉 [Pop into our Slack community!](https://l.linklyhq.com/l/ktO8) We're happy to help with any issue you face or even just to meet you and hear what you're working on.

- Dive deeper into the [Core Concepts](https://docs.bentoml.org/en/latest/concepts/index.html) in BentoML
- Learn how to use BentoML with other ML Frameworks at [Frameworks Guide](https://docs.bentoml.org/en/latest/frameworks/index.html) or check out other [gallery projects](https://github.com/bentoml/gallery)
- Learn more about model deployment options for Bento:
  - [🦄️ Yatai](https://github.com/bentoml/Yatai): Model Deployment at scale on Kubernetes
  - [🚀 bentoctl](https://github.com/bentoml/bentoctl): Fast model deployment on any cloud platform

Bento is designed to be deployed to run efficiently in a variety of different environments.
And there are lots of deployment options and tools as part of the BentoML eco-system, such as 
[Yatai](https://github.com/bentoml/Yatai) and [bentoctl](https://github.com/bentoml/bentoctl) for
direct deployment to cloud platforms.

In this guide, we will show you the most basic way of deploying a Bento, which is converting a Bento
into a Docker image containing the HTTP model server.

Make sure you have docker installed and docker deamon running, and run the following commnand:

```bash
bentoml containerize spam_detector:latest
```

This will build a new docker image with all source code, model files and dependencies in place,
and ready for production deployment. To start a container with this docker image locally, run:

```bash
docker run -p 3000:3000 spam_detector:bufidfqaeg4qz3c4 
```

## What's Next?

- 👉 [Pop into our Slack community!](https://l.linklyhq.com/l/ktO8) We're happy to help with any issue you face or even just to meet you and hear what you're working on.

- Dive deeper into the [Core Concepts](https://docs.bentoml.org/en/latest/concepts/index.html) in BentoML
- Learn how to use BentoML with other ML Frameworks at [Frameworks Guide](https://docs.bentoml.org/en/latest/frameworks/index.html) or check out other [gallery projects](https://github.com/bentoml/gallery)
- Learn more about model deployment options for Bento:
  - [🦄️ Yatai](https://github.com/bentoml/Yatai): Model Deployment at scale on Kubernetes
  - [🚀 bentoctl](https://github.com/bentoml/bentoctl): Fast model deployment on any cloud platform